<a href="https://colab.research.google.com/github/jdmedinatobon/proyectoMachineLearning/blob/master/CrearArchivoDescriptores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#En esta celda se incluyen los comandos necesarios para instalar las librerias requeridas.
#Esta solo se debe ejecutar cada vez que se inicia el Runtime.
!pip install mahotas

ERROR: Could not find a version that satisfies the requirement unzip (from versions: none)
ERROR: No matching distribution found for unzip


In [1]:
#Aqui se importan las librerias necesarias para correr el codigo.
#Antes de correr esta celda se deben ejecutar la anterior para instalar las librerias.
import os
import numpy as np
import nibabel as nib
import csv
import mahotas as mh

%tensorflow_version 2.x
import tensorflow as tf
import time

TensorFlow 2.x selected.


In [0]:
#Se descomprime el archivo con los datos preprocesados
!unzip PruebaDatos.zip #En verdad no es este archivo.

#Indica si existe alguna GPU.
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Archive:  PruebaDatos.zip
   creating: PruebaDatos/
   creating: PruebaDatos/func/
 extracting: PruebaDatos/func/sub-01_task-LTM_run-1_bold.nii.gz  
  inflating: PruebaDatos/func/sub-01_task-LTM_run-1_events.tsv  
Found GPU at: /device:GPU:0


In [0]:
#Es la ruta del archivo raiz de los datos preprocesados.
pathArchivoDatosPreprocesados = "PruebaDatos"

#Estas 2 variables se utilizan para recorrer cada carpeta de los datos.
sub = [0, 0]
run = 1

#Funcion que retorna el nombre del archivo que contiene las imagenes de resonancia magnetica.
#pSub: Indica la persona a la cual se le tomaron las imagenes.
#pRun: Indica el run para una persona. Cada run incluye 180 muestras de imagenes.
def darNombreArchivoImagenes(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_bold.nii.gz"

#Funcion que retorna el nombre del archivo que contiene las etiquetas correspondientes a las imagenes de resonancia magnetica.
#pSub: Indica la persona a la cual se le tomaron las imagenes.
#pRun: Indica el run para una persona. Cada run incluye 180 muestras de imagenes.
def darNombreArchivoEtiquetas(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_events.tsv"

#Funcion que obtiene el conjunto de 30 imagenes de 64x64 utilizados para el proyecto.
def obtenerImagenes(pImagenes):
  
  imagenes = np.zeros((64,64,30,30))

  for indice in range(0,15):
    imagenes[:,:,:,2*indice] = pImagenes[:,:,:,1+indice*13]
    imagenes[:,:,:,2*indice+1] = pImagenes[:,:,:,2+indice*13]

  return imagenes

#Funcion que obtiene las etiquetas y las convierte a su correspondiente numero entero de acuerdo con la siguiente regla:
#object -> 0
#place -> 1
#face -> 2
def leerEtiquetas(pArchivoEtiquetas):
  etiquetas = []

  with open(pArchivoEtiquetas) as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    for row in reader:
      
      etiqueta = row['trial_type']

      if(etiqueta == 'object'):

        etiquetas.append(0)
        etiquetas.append(0)
      elif(etiqueta == 'place'):

        etiquetas.append(1)
        etiquetas.append(1)
      elif(etiqueta == 'face'):

        etiquetas.append(2)
        etiquetas.append(2)
      else:
        print("Error en el formato. Existe una clase distinta a object, place o face")

  return etiquetas

#Funcion que genera una matriz de NumPy con los datos leidos de los archivos.
def importarDatos():
  datosPreprocesados = np.zeros((64,64,30,1800))
  etiquetasPreprocesados = np.zeros(1800)

  contador = 0
  for s1 in range(1,2):#(1,11)

    if(s1 == 10):
      sub[0] = 1
      sub[1] = 0
    else:
      sub[1] = s1

      for r in range(1,2):#(1,7)
        run = r
        archivoImagenes = darNombreArchivoImagenes(sub, run)
        archivoEtiquetas = darNombreArchivoEtiquetas(sub, run)

        imagenes = nib.load(archivoImagenes).get_fdata()
        etiquetas = leerEtiquetas(archivoEtiquetas)

        muestras = obtenerImagenes(imagenes)

        for indice in range(0,30):
            datosPreprocesados[:,:,:,30*contador+indice] = muestras[:,:,:,indice]
            etiquetasPreprocesados[30*contador+indice] = etiquetas[indice]

        contador+=1

    return datosPreprocesados, etiquetasPreprocesados.astype(int)

In [36]:
#En esta linea de codigo se importan los datos y se almacenan en las variables datosPreprocesados (correspondiente a las imagenes) y en etiquetasPreprocesadas (las etiquetas).
print("Importando datos...")
datosPreprocesados, etiquetasPreprocesados = importarDatos()
print("Se importaron correctamente.")

Importando datos...
Se importaron correctamente.


In [0]:
#Calcula los descriptores a partir de un grupo de imagenes de fMRI.
def calcularDescriptores(pImagenes):
  #Faltan los descriptores del histograma
  return mh.features.haralick(pImagenes, ignore_zeros=True, preserve_haralick_bug=False, compute_14th_feature=False, return_mean=True, return_mean_ptp=False, use_x_minus_y_variance=False, distance=1)

#Funcion que retorna el texto de la descripcion del archivo de descriptores.
def darTextoIntroduccion(pEscala):
  texto = "Este archivo incluye los descriptores calculados a partir de los datos preprocesados. \nLos datos brutos fueron obtenidos del \
dataset de imágenes de resonancia magnética funcional que se puede encontrar en https://openneuro.org/datasets/ds001497/versions/1.0.1.\n\
A estos datos se les realizó un preprocesamiento utilizando fmriprep (ESTO TOCA VER SI SI SIRVE AL FIN) y finalmente se calcularon los \
descriptores con una escala de grises de " + str(pEscala) + ".\nEstos descriptores son (EL NUMERO QUE SEA) e incluyen:\nEnergía, etc.... (AGREGAR LOS QUE FALTAN Y ORDENARLOS CORRECTAMENTE).\n\
Finalmente, cada muestra incluye a la clase a la cual pertenece. Existen 3 clases y se indican con un número de 0 a 2 que corresponden a:\n\
objeto -> 0\n\
lugar -> 1\n\
rostro -> 2\n\
energia;kurtosis;skewness;....;clase AGREGAR TODAS LAS VARIABLES ACA EN ORDEN SIGUIENDO LA CONVENCIÓN DE SEPARARLOS CON ;"

  return texto

#Funcion que retorna un string con los descriptores y clases separados por ;.
#Esta cadena sera utilizada para generar el archivo de texto con los descriptores calculados y su etiqueta.
def darLinea(pDescriptores, pEtiqueta):
  separador = ";"
  muestra = []
  return separador.join(pDescriptores.astype(str)) + separador + str(pEtiqueta)

#Funcion que genera el archivo de texto con los datos de los descriptores y su correspondiente etiqueta.
#Este archivo recibe el nombre de datosDescriptores.txt e incluye los descriptores y sus etiquetas separadas por ;
#y una descripcion del archivo.
def crearArchivoDescriptores():
  escala = 256

  print("Generando archivo de descriptores...")
  archivo = open("datosDescriptores.txt", "w+")

  intro = darTextoIntroduccion(escala)
  archivo.write(intro)
  archivo.write("\n")

  start = time.time()
  for i in range(0,30):
    imagenes = datosPreprocesados[:,:,:,i]
    imagenes = escala*(imagenes/imagenes.max())
    imagenes = imagenes.astype(int)

    descriptores = calcularDescriptores(imagenes)
    linea = darLinea(descriptores, etiquetasPreprocesados[i])
    archivo.write(linea)
    archivo.write("\n")
    
  print("Archivo generado exitosamente.")
  print("Tiempo: {} segundos".format(time.time()-start))

  archivo.close()

In [70]:
crearArchivoDescriptores()

Generando archivo de descriptores...
Archivo generado exitosamente.
Tiempo: 3.7933456897735596 segundos
